 # Laplacian learning notebook

In [1]:
import os
import sys
import time
import numpy as np

# -------------Torch-------------------
import torch
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# -------------Graph-------------------
import dgl
import networkx as nx
import matplotlib.pyplot as plt

from utils import set_device
from laplacian import normalized_laplacian

sys.path.insert(0, "lib/")
%load_ext autoreload
%autoreload 2

# MNIST

In [2]:
# Downloads the dataset if not found locally
from Dataset import check_mnist_dataset_exists, datasampler, MNISTDataset
_ = check_mnist_dataset_exists()

train_data, train_labels, test_data, test_labels = datasampler(
    nb_selected_train_data=64, nb_selected_test_data=512
)

#trainset_ref = MNISTDataset(train_data, train_labels)
trainset_8 =  MNISTDataset(train_data, train_labels, lattice_type=1)
#testset = MNISTDataset(test_data, test_labels)
testset =  MNISTDataset(test_data, train_labels, lattice_type=1)

# Model import

In [3]:
from model import Classifier, SmallCheb

from utils import collate

In [4]:
# model_parameters
in_features = 1
first_conv_out = 32
second_conv_out = 50
hidden_layer_classifier = 512
k = 10
n_classes = testset.num_classes

reference_net = Classifier(
    in_features, first_conv_out, second_conv_out, hidden_layer_classifier, n_classes, k
)

# LOAD MODEL
from utils import load_model
reference_net.load_state_dict(load_model('Model_6000_20ep'))
reference_net.eval()

Classifier(
  (layers): ModuleList(
    (0): Chebyconv(
      (apply_mod): NodeApplyModule(
        (linear): Linear(in_features=10, out_features=32, bias=True)
      )
    )
    (1): Chebyconv(
      (apply_mod): NodeApplyModule(
        (linear): Linear(in_features=320, out_features=50, bias=True)
      )
    )
  )
  (classify): Sequential(
    (0): Linear(in_features=39200, out_features=512, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=512, out_features=10, bias=True)
  )
)

## Laplacian model def

In [5]:
from utils import npsparse_to_torch


class boostlaplace(nn.Module):
    '''
    One layer, one module, one parameter
    '''

    def __init__(self, in_dim, out_dim):
        super(boostlaplace, self).__init__()
        self.fc1 = nn.Linear(in_dim, out_dim, bias=False)
        # Adding Bias is going to modify the whole laplacian and 
        # not the non-zero values. We can experiment with bias in a more 
        # complex version later.

    def forward(self, L):
        n, m = L.size()
        L = self.fc1(L.to_dense().view(-1, 1))
        #print('L2', L, L.size())
        L = L.view(n, m)
        #print('L3', L, L.size())
        return L


net = boostlaplace(1, 1)

# Training time!

In [6]:
# Use PyTorch's DataLoader and the collate function
# defined before.
b_size = 16
data_loader = DataLoader(trainset_8, batch_size=b_size,
                         shuffle=True, collate_fn=collate)

# Distance loss
#loss_func = nn.MSELoss()

# Stochastic optimiser
#optimizer = optim.SGD(net.parameters(), lr=0.005)

loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.005)

epoch_losses = []
running_vals = []

In [ ]:
from laplacian import normalized_laplacian

for epoch in range(2):
    epoch_loss = 0
    t0 = time.time()
    for iter, (bg, label, signal) in enumerate(data_loader):
        if iter == 0:
            #L = normalized_laplacian(bg)
            L_8 = normalized_laplacian(bg)
            L_8_variable = net(L_8)
        
        prediction = reference_net(bg, signal, L_8_variable)

        if iter == 0:
            #print("prediction time:", time.time() - t)
            t = time.time()
            print("One Prediction:\n", prediction[0], 'len:',
                  prediction.size(), '\n label:', label[0])  # DEBUG
            
        loss = loss_func(prediction, label)
        
        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()

        epoch_loss += loss.detach().item()

    epoch_loss /= iter + 1
    print("weight",net.fc1.weight)
    running_vals.append(net.fc1.weight)
    print(
        "Epoch {}, loss {:.4f}, in {:.2f}(s) ".format(
            epoch, epoch_loss, time.time() - t0
        )
    )
    epoch_losses.append(epoch_loss)

One Prediction:
 tensor([ 15.9075,  -1.7827,   3.6574,  17.3755, -29.2694,  22.5588,  21.7306,
         -5.8322,  40.9078,  31.0212], grad_fn=<SelectBackward>) len: torch.Size([16, 10]) 
 label: tensor(1)


In [ ]:
plt.title("cross entropy averaged over minibatches")
plt.plot(epoch_losses)
plt.show()

In [ ]:
plt.imshow(L_8_variable.detach().numpy(),cmap='viridis')
plt.colorbar()

In [ ]:
net.fc1.weight

# RE-build the classifier

In [ ]:
# model_parameters
loaded_net = Classifier(1, 32, 50, 512, testset.num_classes, 10)

# LOAD MODEL
from utils import load_model
loaded_net.load_state_dict(load_model('Model_6000_20ep'))

# Test


In [ ]:
import sklearn
from sklearn.metrics import classification_report

def test_func(data_loader):
    loaded_net.eval()
    probs_Y = torch.Tensor()
    test_Y = torch.Tensor().int()
    for iter, (bg, label, signal) in enumerate(data_loader):
        if iter == 0:
            L = normalized_laplacian(bg)
        probs_Y = torch.cat((probs_Y, torch.softmax(reference_net(bg, signal, L), 1)), 0)
        test_Y = torch.cat((test_Y, label.int()),0)

    sampled_Y = torch.multinomial(probs_Y, 1)
    argmax_Y = torch.max(probs_Y, 1)[1].view(-1, 1)
    return sklearn.metrics.classification_report(test_Y, argmax_Y, digits=4)

data_loader_test = DataLoader(testset, batch_size=b_size,
                         shuffle=False, collate_fn=collate)
print(test_func(data_loader_test))